<a href="https://colab.research.google.com/github/Francesca-Rossi/ITA_university_advisor/blob/main/GetDataFromGoogleSheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importazione librerie

In [1]:
import gspread 
import nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np
import os
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import drive
from nltk.corpus import stopwords
from nltk.stem.snowball import ItalianStemmer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#COLLEGAMENTO CON IL GOOGLE SHEET 

In [2]:
scopes = ["https://spreadsheets.google.com/feeds",
                  "https://www.googleapis.com/auth/spreadsheets",
                  "https://www.googleapis.com/auth/drive",
                  "https://www.googleapis.com/auth/drive"]

In [3]:
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/big data proj")
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['GoogleSheetCredentials.json']

In [4]:
cred = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/My Drive/Colab Notebooks/big data proj/GoogleSheetCredentials.json", scopes)

In [5]:
gclient = gspread.authorize(cred)

In [6]:
sheet = gclient.open_by_url("https://docs.google.com/spreadsheets/d/1scMc0UlNC1pdZNow5k3pLhofcW6j4ekXyxYlNiBcueI/edit?usp=sharing")

# RECUPERO FOGLI E SALVATAGGIO IN DATAFRAME

In [7]:
students_data = sheet.worksheet('Laureando').get_all_records()
graduate_data = sheet.worksheet('Laureato').get_all_records()

In [8]:
df_students = pd.DataFrame(students_data)
#print(df_students)

In [9]:
df_graduate = pd.DataFrame(graduate_data)
#print(graduate_df)

#GESTIONE VALORI MANCANTI NELLA COLONNA UNIVERSITA'

##Studenti

###1 - Sostituzione celle vuote con valore 'Nan'

In [10]:
df_students=df_students.replace('',np.NaN)
df_students['university']=df_students['university'].replace('altro', np.NaN ) #necessario per effettuare merge tra 2 colonne
df_students.shape

(719, 89)

In [11]:
#TODO remove [fra]
df_students.isnull().sum()#codice di controllo

age                        101
gender                      90
hometown                     7
home_province                7
home_region                  9
                          ... 
prev_degree_couse_class    680
expectations_class         128
expectations_no_class      586
change_why_class           681
decision_choice_class      130
Length: 89, dtype: int64

In [12]:
#TODO remove [fra]
df_students[['university', 'other_uni']].head(50) #codice di controllo

,university,other_uni
0,Politecnico di MILANO,NaN
1,Università degli Studi di NAPOLI Federico II,NaN
2,Università degli Studi di ROMA La Sapienza,NaN
3,Università degli Studi di ROMA La Sapienza,NaN
4,Università degli Studi di CAGLIARI,NaN
5,Università degli Studi di PARMA,NaN
6,NaN,"Civica Scuola di Cinema ""Luchino Visconti"""
7,Università degli Studi di BOLOGNA,NaN
8,Università degli Studi di PARMA,NaN
9,Università degli Studi di PARMA,NaN


###2- Merge tra la colonna university e other_uni per unire i valori mancanti in university

In [13]:
df_students['university']=df_students['university'].combine_first(df_students['other_uni'])
#df_students['university'].head(50)

0                                Politecnico di MILANO
1         Università degli Studi di NAPOLI Federico II
2           Università degli Studi di ROMA La Sapienza
3           Università degli Studi di ROMA La Sapienza
4                   Università degli Studi di CAGLIARI
5                      Università degli Studi di PARMA
6           Civica Scuola di Cinema "Luchino Visconti"
7                    Università degli Studi di BOLOGNA
8                      Università degli Studi di PARMA
9                      Università degli Studi di PARMA
10    Università degli Studi di MODENA e REGGIO EMILIA
11                   Università degli Studi di PALERMO
12          Università degli Studi di ROMA La Sapienza
13                    Università degli Studi di PADOVA
14                    Università degli Studi di GENOVA
15                    Università degli Studi di MILANO
16                     Università degli Studi di PARMA
17                  Università degli Studi di CAGLIARI
18        

### 3 - Eliminazione delle righe che nonostante il merge presentano valori nulli nella colonna university

In [14]:
df_students = df_students[df_students['university'].notna()]
df_students.shape

(696, 89)

##Laureati

### 1- Sostituzione delle celle vuote con il valore 'Nan'

In [15]:
df_graduate=df_graduate.replace('',np.NaN)
df_graduate['university']=df_graduate['university'].replace('altro', np.NaN ) #per poter fare il merge tra due colonne
df_graduate.shape

(61, 85)

In [16]:
#TODO remove [fra]
df_graduate.isnull().sum() #codice di controllo

age                       2
gender                    3
hometown                  2
home_province             1
home_region               2
                         ..
redo_choice_why_class     3
expectations_class       21
expectations_no_class    53
change_why_class         60
decision_choice_class    14
Length: 85, dtype: int64

In [17]:
#TODO remove [fra]
df_graduate[['university', 'other_uni']].head(50) #codice di controllo

,university,other_uni
0,Università degli Studi di PARMA,NaN
1,Università degli Studi di PARMA,NaN
2,Università degli Studi di PARMA,NaN
3,Università degli Studi di PARMA,NaN
4,Università degli Studi di VERONA,NaN
5,Università degli Studi di MODENA e REGGIO EMILIA,NaN
6,Università degli Studi di PARMA,NaN
7,Università degli Studi di NAPOLI Federico II,NaN
8,Università degli Studi di PADOVA,NaN
9,Università degli Studi di ROMA Foro Italico,NaN


### 2- Merge tra colonna university e other_uni

In [18]:
df_graduate['university']=df_graduate['university'].combine_first(df_graduate['other_uni'])
#df_graduate['university'].head(50)

###3- Eliminazione delle righe che contengono ancora 'Nan' nella colonna university

In [19]:
df_graduate = df_graduate[df_graduate['university'].notna()]
df_graduate.shape

(57, 85)

#GESTIONE VALORI MANCANTI NELLA COLONNA CORSO DI LAUREA

In [20]:
#acronimi
slang_degree_course={"ctf":"chimica e tecnologie farmaceutiche",
       "clemi": "economia e marketing internazionale", 
       "sztpa": "scienze zootecniche e tecnologie delle produzioni animali", 
       "smid": "statistica matematica e trattamento informatico dati", 
       "clei": "economia internazionale",
       "clem": "economia e marketing", 
       "dams": "discipline delle arti della musica e dello spettacolo",
       "ctc": "chimica e tecnologie chimiche",
       "pmts": "planning management of tourism systems",
       "eifi":  "lingue moderne comunicazione cooperazione internazionale",
       "lms":  "letteratura musica spettacolo"
        }
#caratteri speciali
special_char=[',',"’","'",'<', '>', '.', '-', '_', '/', ': ', ' & ']

## Studenti

### Elimino righe che hanno valore nullo nel campo degree_course

In [21]:
df_students = df_students[df_students['degree_course'].notna()]
df_students.shape

(690, 89)

### Lavoro su una nuova colonna creata ad hoc

In [22]:
df_students['clean_degree_course']=df_students['degree_course']

###Pulisco i dati

In [23]:
#metto in minuscolo
df_students['clean_degree_course']=df_students['clean_degree_course'].apply(lambda x: x.lower())
#df_students['degree_course'].head(50)

####Riconoscimento della lingua...non funziona

In [24]:
#import langid
#langid.set_languages(['it', 'en'])  # ISO 639-1 codes
#def detect_language(text):
  #lang, score = langid.classify(text)
  #return lang
#df_students['language_degree_course']=df_students['clean_degree_course'].apply(lambda x: detect_language(x))
#df_students[['language_degree_course', 'clean_degree_course']][df_students['language_degree_course'].str.contains('en')]

In [25]:
for key in slang_degree_course.keys():
  df_students.loc[df_students['clean_degree_course'].str.contains(key), 'clean_degree_course'] = slang_degree_course[key]
#df_students[['clean_degree_course','degree_course']].head(50)

In [26]:
df_students.loc[df_students['clean_degree_course'].str.contains('medicina') & ~df_students['clean_degree_course'].str.contains('veterinaria'), 'clean_degree_course'] = 'medicina e chirurgia'
df_students.loc[df_students['clean_degree_course'].str.contains('informatica') & df_students['clean_degree_course'].str.contains('elettronica') & df_students['clean_degree_course'].str.contains('telecomunicazioni') , 'clean_degree_course'] = 'ingegneria informatica elettronica e telecomunicazioni'
df_students.loc[df_students['clean_degree_course'].str.contains('comunicazione e media per le industrie creative'), 'clean_degree_course'] = 'comunicazione e media contemporanei per le industrie creative'
df_students.loc[df_students['clean_degree_course'].str.contains('elettronica') & ~df_students['clean_degree_course'].str.contains('ingegneria' )& ~df_students['clean_degree_course'].str.contains('ing'), 'clean_degree_course'] = 'ingegneria elettronica'
df_students.loc[df_students['clean_degree_course'].str.contains('interfacce e tecnologie della comunicazion'), 'clean_degree_course'] = 'interfacce uomo macchina e tecnologie della comunicazione'
df_students.loc[df_students['clean_degree_course'].str.contains('letteratura musica e spettacolo'), 'clean_degree_course'] = 'letteratura musica e spettacolo'
#df_students['clean_degree_course']e 

In [27]:
#eliminare i corso di laurea
df_students['clean_degree_course']=df_students['clean_degree_course'].str.replace('corso di laurea', '')

In [28]:
#rimozione di caratteri speciali
for i in range(0, len(special_char)):
  df_students['clean_degree_course']=df_students['clean_degree_course'].str.replace(special_char[i], ' ')
#df_students['degree_course'].head(50)

In [29]:
df_students['clean_degree_course']=df_students['clean_degree_course'].str.replace(r'\sl.*[0-9]+', '') #lm19 , l-12
df_students['clean_degree_course']=df_students['clean_degree_course'].str.replace('ingegneria ', 'ing ')
df_students['clean_degree_course']=df_students['clean_degree_course'].str.replace('ingeneria ', 'ing ')
df_students['clean_degree_course']=df_students['clean_degree_course'].str.replace(' ciclo unico', '')
df_students['clean_degree_course']=df_students['clean_degree_course'].str.replace(r'\scurriculum\s.*', '')
df_students['clean_degree_course']=df_students['clean_degree_course'].str.replace(r'indirizzo.*', '')
df_students['clean_degree_course']=df_students['clean_degree_course'].str.replace(r'[(].*[)]*', '')
#df_students.loc[df_students['clean_degree_course'].str.contains(r'l.*[0-9]+'), 'clean_degree_course'] 

In [30]:
#eliminazione spazi bianchi prima e dopo una stringa
df_students['clean_degree_course']=df_students['clean_degree_course'].str.strip()
#df_students['degree_course'].head(50)

In [31]:
#elimino doppi spazi bianchi
df_students['clean_degree_course']=df_students['clean_degree_course'].str.replace('  ', ' ')

###Eliminazione stop words

In [32]:
# Split the sentences to lists of words.
df_students['clean_degree_course'] = df_students['clean_degree_course'].str.split()
df_students['clean_degree_course'] 

0                          [ing, aerospaziale]
1                     [economia, e, commercio]
2                     [medicina, e, chirurgia]
3                        [psicologia, clinica]
4                     [medicina, e, chirurgia]
                        ...                   
713                           [ing, biomedica]
714                      [scienze, biologiche]
715                              [informatica]
716                                   [fisica]
718    [chimica, e, tecnologie, farmaceutiche]
Name: clean_degree_course, Length: 690, dtype: object

In [33]:
stop_ita = stopwords.words('italian')
df_students['clean_degree_course']=df_students['clean_degree_course'].apply(lambda x: [item for item in x if item not in stop_ita])
#stop_ita

In [34]:
stop_en = stopwords.words('english')
df_students['clean_degree_course']=df_students['clean_degree_course'].apply(lambda x: [item for item in x if item not in stop_en])
#stop_en

###Estrazione della radice-stemming

In [35]:
#stemmer = ItalianStemmer()
#df_students['clean_degree_course'] = df_students['clean_degree_course'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
#df_students['clean_degree_course'] 

In [36]:
#Ricompongo le frasi
df_students['clean_degree_course']=df_students['clean_degree_course'].apply(' '.join)

In [57]:
#TODO remove fra (codice di prova)
df_students[['clean_degree_course','degree_course']].head(50)

,clean_degree_course,degree_course
0,ing aerospaziale,ingegneria aerospaziale
1,economia commercio,Economia e commercio
2,medicina chirurgia,Medicina
3,psicologia clinica,Psicologia clinica
4,medicina chirurgia,Medicina e chirurgia
5,infermieristica,Infermieristica
6,montaggio cinematografico televisivo,Montaggio Cinematografico e Televisivo
7,lingua cultura italiane stranieri,Lingua e cultura italiane per stranieri
8,professioni sanitarie,Professioni Sanitarie
9,matematica,Matematica


In [38]:
degree_course=np.unique(df_students['clean_degree_course'])
print(len(degree_course))
print(degree_course)

246
['advanced sportscar manufacturing' 'amministrazione direzione aziendale'
 'amministrazione finanza' 'analisi processi sociali' 'archeologia'
 'architettura' 'arti scienze spettacolo'
 'artificial intelligence robotics' 'automation control engineering'
 'automazione industriale' 'banca finanza' 'beni artistici spettacolo'
 'beni culturali' 'beni culturali spettacolo' 'bioingegneria' 'biologia'
 'biologia ecologia marina' 'biologia molecolare'
 'biologia molecolare applicata nutrizione'
 'biologia sperimentale applicata' 'biomedical engineering'
 'bionanotecnologie' 'biotecnologie' 'biotecnologie mediche'
 'biotecnologie sicurezza qualità agro alimentare' 'chimica'
 'chimica clinica forense sport' 'chimica industriale'
 'chimica tecnologia farmaceutiche' 'chimica tecnologie chimiche'
 'chimica tecnologie farmaceutiche' 'civiltà lingue straniere moderne'
 'computer science' 'comunicazione' 'comunicazione impresa risorse umane'
 'comunicazione media contemporanei industrie creative'
 

#UNIONE UNIVERSITA'-LAUREA

In [39]:
#TODO: remove francesca
#df_students['uni-course'] = df_students['university'].str.cat(df_students['degree_course'],sep=" ")
#df_students['uni-course']

In [40]:
#df_graduate['uni-course'] = df_graduate['university'].str.cat(df_graduate['degree_course'],sep=" ")
#df_graduate['uni-course']

#DATI TRIENNALE E MAGISTRALE CICLO UNICO

<p>Selezionare le righe secondo una determinata condizione</p>
<a href="https://chrisalbon.com/python/data_wrangling/pandas_selecting_rows_on_conditions/"> https://chrisalbon.com/python/data_wrangling/pandas_selecting_rows_on_conditions/</a>

In [59]:
df_students_first_degree = df_students.query('study_type == "Triennale" | study_type == "Magistrale_unico"')
#df_students_first_degree.isnull().sum()
df_students_first_degree.shape
#df_students_first_degree

(552, 90)

In [42]:
df_graduate_first_degree = df_graduate.query('study_type == "Triennale" | study_type == "Magistrale_unico"')
#df_graduate_first_degree
df_graduate_first_degree.shape

(27, 85)

# DATI MAGISTRALI

In [43]:
df_students_second_degree = df_students.query('study_type == "Magistrale"')
#df_students_second_degree
df_students_second_degree.shape

(138, 90)

In [44]:
df_graduate_second_degree = df_graduate.query('study_type == "Magistrale"')
#df_graduate_second_degree
df_graduate_second_degree.shape

(30, 85)

# ANALISI SCUOLA SUPERIORE -TODO

##Preparazione dataset

Queste righe di codice mi permettono di selezionare solo le colonne che ci interessano per l analisi e unire 2 dataframe. Devo fare così in quanto gli studenti e i laureati non hanno lo stesso numero di colonne e quindi non possono essere uniti insieme

In [45]:
#Selezionare da colonna a colonna tutti le righe
#df_students_second_degree.loc['degree_course':'study_type']

In [46]:
#selezionare da solo determinate colonne
df_high_school_analysis=df_students_first_degree[['university','degree_course','high_school']]
df_high_school_analysis

,university,degree_course,high_school
0,Politecnico di MILANO,ingegneria aerospaziale,Liceo Scientifico
1,Università degli Studi di NAPOLI Federico II,Economia e commercio,Liceo Scientifico
2,Università degli Studi di ROMA La Sapienza,Medicina,Liceo Classico
4,Università degli Studi di CAGLIARI,Medicina e chirurgia,Liceo Classico
5,Università degli Studi di PARMA,Infermieristica,Liceo Classico
...,...,...,...
713,Università degli Studi di GENOVA,Ingegneria biomedica,Istituto tecnico industriale
714,Università degli Studi di PERUGIA,Scienze biologiche,Liceo Linguistico
715,Università degli Studi di ROMA La Sapienza,Informatica,altro
716,Università degli Studi di PARMA,Fisica,Liceo Classico


In [47]:
df_high_school_analysis.append(df_graduate_first_degree[['university','degree_course','high_school']])

,university,degree_course,high_school
0,Politecnico di MILANO,ingegneria aerospaziale,Liceo Scientifico
1,Università degli Studi di NAPOLI Federico II,Economia e commercio,Liceo Scientifico
2,Università degli Studi di ROMA La Sapienza,Medicina,Liceo Classico
4,Università degli Studi di CAGLIARI,Medicina e chirurgia,Liceo Classico
5,Università degli Studi di PARMA,Infermieristica,Liceo Classico
...,...,...,...
52,Università degli Studi di PAVIA,ingegneria,Istituto tecnico geometri
54,Università degli Studi di ROMA La Sapienza,Medicina e chirurgia,Liceo Scientifico
57,Libera Università Vita Salute San Raffaele MILANO,Scienze motorie,Istituto tecnico economico
58,Università degli Studi di GENOVA,Servizio Sociale,Liceo Scientifico


#ANALISI MATERIE -TODO

##Preparazione dataset

In [48]:
#seleziono le colonne che mi servono
df_subject_area_analysis=df_students_first_degree[['university','degree_course','subject_area']]
df_subject_area_analysis

,university,degree_course,subject_area
0,Politecnico di MILANO,ingegneria aerospaziale,"Matematica,meccanica,fisica"
1,Università degli Studi di NAPOLI Federico II,Economia e commercio,"economia,diritto"
2,Università degli Studi di ROMA La Sapienza,Medicina,"Anatomia, fisiologia, patologia"
4,Università degli Studi di CAGLIARI,Medicina e chirurgia,"Anatomia, patologia,"
5,Università degli Studi di PARMA,Infermieristica,"Anatomia, biochimica, fisiologia, patologia, f..."
...,...,...,...
713,Università degli Studi di GENOVA,Ingegneria biomedica,"Fisiologia, elettronica, meccanica"
714,Università degli Studi di PERUGIA,Scienze biologiche,"Matematica,chimica organica,chimica inorganica..."
715,Università degli Studi di ROMA La Sapienza,Informatica,"Algoritmi, programmazione"
716,Università degli Studi di PARMA,Fisica,"Matematica,fisica"


In [49]:
df_subject_area_analysis=df_subject_area_analysis.append(df_graduate_first_degree[['university','degree_course','subject_area']])

##Gestione valori mancanti: da rivedere

 Blocco con rientro



In [50]:
#numero di colonne con valori mancanti
print("Columns with Missing Values:",df_subject_area_analysis.columns[df_subject_area_analysis.isnull().any()].tolist())

Columns with Missing Values: ['subject_area']


In [51]:
#numero di righe con valori mancanti
print("Number of rows with Missing Values::",df_subject_area_analysis.isnull().any(axis = 1).sum()) #???

Number of rows with Missing Values:: 14


In [52]:
#stampa le righe con valori mancanti
null_data = df_subject_area_analysis[df_subject_area_analysis.isnull().any(axis=1)]
null_data

,university,degree_course,subject_area
71,Università degli Studi di CAGLIARI,Medicina e Chirurgia,NaN
126,Università degli Studi di MILANO-BICOCCA,Infermieristica,NaN
309,Università degli Studi di PARMA,Economia,NaN
372,Università degli Studi di MODENA e REGGIO EMILIA,Economia e marketing internazionale,NaN
396,Università degli Studi di CAGLIARI,Medicina e chirurgia,NaN
412,Università degli Studi di MODENA e REGGIO EMILIA,Ingegneria informatica,NaN
523,Università degli Studi di GENOVA,Scienze Biologiche,NaN
571,Università degli Studi di PARMA,Ingegneria dei Sistemi Informativi,NaN
585,Università degli Studi di CAGLIARI,Medicina e Chirurgia,NaN
623,Università degli Studi di PARMA,Ingegneria Informatica,NaN


In [53]:
#university = np.unique(df_subject_area_analysis['university'])
#print("Università presenti:", len(university))
#print(university)

https://pythoninoffice.com/use-fuzzy-string-matching-in-pandas/
https://stackoverflow.com/questions/13636848/is-it-possible-to-do-fuzzy-match-merge-with-python-pandas
https://www.youtube.com/watch?v=uxHnMG1Jz00
https://towardsdatascience.com/fuzzywuzzy-find-similar-strings-within-one-column-in-a-pandas-data-frame-99f6c2a0c212

In [54]:
#!pip install fuzzywuzzy
#from fuzzywuzzy import fuzz
#from fuzzywuzzy import process
#df_subject_area_analysis_1 =df_subject_area_analysis.copy()
#df_subject_area_analysis_1
#matches= df_subject_area_analysis['degree_course'].apply(lambda x: process.extractOne(x, df_subject_area_analysis_1['degree_course'].to_list(),score_cutoff=80))
#df_subject_area_analysis[['degree_course']]

In [55]:
#df=pd.DataFrame(list(matches), columns=['degree', 'match'])
#df

In [56]:
#df_subject_area_analysis['degree_course']=df['degree']
#df_subject_area_analysis['degree_course']

https://stackoverflow.com/questions/49716597/group-similar-words-sentences-in-pandas-column